In [1]:
#| default_exp misc

In [2]:
#| export
import time
from torch_snippets.logger import Info
from torch_snippets.inspector import inspect

In [3]:
#| export
#| hide

class Timer:
    def __init__(self, N):
        "print elapsed time every iteration and print out remaining time"
        "assumes this timer is called exactly N times or less"
        self.start = time.time()
        self.N = N
        self.ix = 0

    def __call__(self, ix=None, info=None):
        ix = self.ix if ix is None else ix
        info = "" if info is None else f"{info}\t"
        elapsed = time.time() - self.start
        speed = elapsed / (ix + 1)
        unit = 's/iter'
        if speed < 1:
            speed = 1 / speed
            unit = 'iters/s'
        print(
            "{}{}/{} ({:.2f}s - {:.2f}s remaining - {:.2f} {}){}".format(
                info, ix + 1, self.N, elapsed, (self.N - ix) * (elapsed / (ix + 1)), speed, unit, " "*10
            ),
            end="\r",
        )
        self.ix += 1

In [4]:
N = 100
t = Timer(N)
for i in range(N):
    time.sleep(0.01)
    t()
    if i == 50:
        print()

51/100 (0.58s - 0.57s remaining - 88.39 iters/s)          


In [5]:
#| export
#| hide

def timeit(func):
    def inner(*args, **kwargs):
        s = time.time()
        o = func(*args, **kwargs)
        Info(f"{time.time() - s:.2f} seconds to execute `{func.__name__}`")
        return o

    return inner


def io(func):
    def inner(*args, **kwargs):
        s = time.time()
        o = func(*args, **kwargs)
        Info(f"Args: {inspect(args)}\nKWargs: {inspect(kwargs)}\nOutput: {inspect(o)}")
        return o

    return inner


In [6]:
@io
@timeit
def foo(a, b=None):
    if b is None:
        return a+1
    else:
        time.sleep(2)
        return a+b
    

foo(10)
foo(10, b=20)

[03/20/23 18:59:31] INFO     0.00 seconds to execute `foo`                                    ]8;id=951613;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_9211/3733636588.py\3733636588.py]8;;\:]8;id=221759;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_9211/3733636588.py#inner:8\inner:8]8;;\

══════════════════════════════════════════════════════════════════

tuple of 1 items

int: 10

══════════════════════════════════════════════════════════════════

...

══════════════════════════════════════════════════════════════════

int: 30

══════════════════════════════════════════════════════════════════

[03/20/23 18:59:34] INFO     Args: None                                                      ]8;id=156524;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_9211/3733636588.py\3733636588.py]8;;\:]8;id=522887;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_9211/3733636588.py#inner:18\inner:18]8;;\
                             KWargs: None                                                                          
                             Output: None                                                                          

30